In [2]:
import pandas as pd
from dateutil import parser
import matplotlib.pyplot as plt
import numpy as np

In [24]:
admission_table = pd.read_csv('../data/ADMISSIONS.csv')

In [25]:
patient_table = pd.read_csv('../data/PATIENTS.csv')

In [26]:
# Remove patients second admittion record
admission_table.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [22]:
patient_table.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [5]:
admission_table['ADMITTIME']=admission_table['ADMITTIME'].map(lambda x: parser.parse(x))
sub_hops_admission = admission_table.groupby(['SUBJECT_ID'])['ADMITTIME'].unique()
sub_hops_admission = sub_hops_admission.map(lambda x: sorted(x)[0])

In [27]:
admission_table = admission_table.groupby(by = 'SUBJECT_ID').first()

In [29]:
admission_table['SUBJECT_ID'] = admission_table.index

In [30]:
admission_with_DOB = pd.merge(admission_table, patient_table[['SUBJECT_ID','DOB']], how='inner', on='SUBJECT_ID')

In [33]:
admission_with_DOB['ADMITTIME'] = pd.to_datetime(admission_with_DOB['ADMITTIME'])
admission_with_DOB['DOB'] = pd.to_datetime(admission_with_DOB['DOB'])
admission_with_DOB['DISCHTIME'] = pd.to_datetime(admission_with_DOB['DISCHTIME'])
admission_with_DOB['DEATHTIME'] = pd.to_datetime(admission_with_DOB['DEATHTIME'])

In [34]:
admission_with_DOB['Age'] =  (admission_with_DOB['ADMITTIME'] - admission_with_DOB['DOB'])

In [35]:
admission_with_DOB['Age'] = admission_with_DOB['Age'].map(lambda x : x.days)

In [36]:
adult = admission_with_DOB[admission_with_DOB['Age']>=18*365]

In [37]:
print('Excluded %d minority patients' %(len(admission_with_DOB['SUBJECT_ID'].unique()) - len(adult['SUBJECT_ID'].unique())))

Excluded 9962 minority patients


In [39]:
adult.head()

,ROW_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,SUBJECT_ID,DOB,Age
1,2,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,...,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1,3,2025-04-11,27950
2,3,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,NaN,...,SINGLE,WHITE,2191-03-15 13:10:00,2191-03-16 01:10:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1,4,2143-05-12,17475
4,5,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,...,MARRIED,WHITE,NaN,NaN,CHRONIC RENAL FAILURE/SDA,0,1,6,2109-06-21,24084
7,8,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicaid,NaN,...,NaN,UNKNOWN/NOT SPECIFIED,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,1,9,2108-01-26,15263
9,10,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,NaN,...,MARRIED,WHITE,2178-04-15 20:46:00,2178-04-16 06:53:00,BRAIN MASS,0,1,11,2128-02-22,18316


In [40]:
adult_with_death_time = adult[~ pd.isnull(adult.DEATHTIME)] 
adult_without_death_time = adult[pd.isnull(adult.DEATHTIME)] 

In [41]:
adult_with_death_time.head()

,ROW_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,SUBJECT_ID,DOB,Age
7,8,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicaid,NaN,...,NaN,UNKNOWN/NOT SPECIFIED,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,1,9,2108-01-26,15263
10,11,112213,2104-08-07 10:15:00,2104-08-20 02:57:00,2104-08-20 02:57:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,DEAD/EXPIRED,Medicare,NaN,...,MARRIED,WHITE,NaN,NaN,PANCREATIC CANCER/SDA,1,1,12,2032-03-24,26433
17,19,109451,2134-09-11 12:17:00,2134-09-24 16:15:00,2135-02-08 02:08:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,...,MARRIED,WHITE,2134-09-11 09:22:00,2134-09-11 22:30:00,CONGESTIVE HEART FAILURE,0,1,21,2047-04-04,31936
26,30,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,...,MARRIED,WHITE,NaN,NaN,STATUS EPILEPTICUS,1,1,31,2036-05-17,26394
53,60,176332,2118-06-17 14:40:00,2118-07-20 12:30:00,2119-02-03 01:35:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,NaN,...,MARRIED,WHITE,NaN,NaN,NON-HODGKIN'S LYMPHOMIA\BONE MARROW TRANSPLANT,0,1,61,2063-10-21,19962


In [42]:
adult_with_death_time = adult_with_death_time[(adult_with_death_time.ADMITTIME < adult_with_death_time.DISCHTIME)\
                                              & (adult_with_death_time.DEATHTIME >= adult_with_death_time.DISCHTIME)]

In [33]:
#  Verstion 2 death after first discharge
adult_with_death_time[(adult_with_death_time.DEATHTIME > adult_with_death_time.DISCHTIME)] = np.nan

In [43]:
adult = pd.concat([adult_with_death_time,adult_without_death_time])

In [44]:
adult_ID =  adult[['SUBJECT_ID','DEATHTIME']]

In [45]:
adult_ID.to_csv('id_inhospital_deathtime.csv')

In [46]:
print('In total Excluded %d patients' \
      %(len(admission_table['SUBJECT_ID'].unique()) - len(adult_ID['SUBJECT_ID'].unique())))

In total Excluded 10038 patients
